In [42]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/GitHub/dynamics-simulation-16-715/lecture-notebooks/Lecture 12/Project.toml`


In [43]:
using LinearAlgebra
using ForwardDiff
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [44]:
g = 9.81
w = 10 #width of gap
ℓ = 15 #length of cable
m = 10 #mass of cable
n = 25 #number of discrete segments

Δx = w/n #width of each discrete segment
ρ = m/ℓ #mass density

0.6666666666666666

In [45]:
#Total cable length
function S(y,ẏ)
    s = 0.0
    for k = 1:(n-1)
        s += sqrt(1 + ẏ[k]^2)*Δx
    end
    return s
end

S (generic function with 1 method)

In [46]:
#Potential energy
function U(y,ẏ)
    u = 0.0
    for k = 1:(n-1)
        ym = 0.5*(y[k+1]+y[k])
        u += ρ*g*ym*sqrt(1 + ẏ[k]^2)*Δx
    end
    return u
end

U (generic function with 1 method)

In [53]:
#End points
y1 = 0.0
yn = -2.0

-2.0

In [54]:
#Objective and constraint functions for IPOPT

function objective(z)
    y = z[1:n]
    ẏ = z[(n+1):end]
    U(y,ẏ)
end

function constraint!(c,z)
    y = z[1:n]
    ẏ = z[(n+1):end]
    
    c .= [
        y[1] - y1 #initial point
        y[end] - yn #final point
        S(y,ẏ) - ℓ #length constraint
        y[2:end] .- y[1:(end-1)] .- ẏ.*Δx #velocity constraints
    ]
    
    return nothing
    
end

constraint! (generic function with 1 method)

In [55]:
#Boilerplate code to interface with IPOPT
struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    primal_bounds
    constraint_bounds
    hessian_lagrangian::Bool
end

function ProblemMOI(n_nlp,m_nlp;
        idx_ineq=(1:0),
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        primal_bounds=primal_bounds(n_nlp),
        constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
        hessian_lagrangian=false)

    ProblemMOI(n_nlp,m_nlp,
        idx_ineq,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        primal_bounds,
        constraint_bounds,
        hessian_lagrangian)
end

function primal_bounds(n)
    x_l = -Inf*ones(n)
    x_u = Inf*ones(n)
    return x_l, x_u
end

function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)
    c_l[idx_ineq] .= -Inf

    c_u = zeros(m)
    return c_l, c_u
end

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    objective(x)
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    ForwardDiff.gradient!(grad_f,objective,x)
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,g,x)
    constraint!(g,x)
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    ForwardDiff.jacobian!(reshape(jac,prob.m_nlp,prob.n_nlp), constraint!, zeros(prob.m_nlp), x)
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac

function solve(x0,prob::MOI.AbstractNLPEvaluator;
        tol=1.0e-6,c_tol=1.0e-6,max_iter=10000)
    x_l, x_u = prob.primal_bounds
    c_l, c_u = prob.constraint_bounds

    nlp_bounds = MOI.NLPBoundsPair.(c_l,c_u)
    block_data = MOI.NLPBlockData(nlp_bounds,prob,true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iter
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        xi = MOI.SingleVariable(x[i])
        MOI.add_constraint(solver, xi, MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, xi, MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # Solve the problem
    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res
end

solve (generic function with 1 method)

In [56]:
#initial guess
z_guess = zeros(2*n-1);

In [57]:
#Solve with IPOPT
n_nlp = 2*n-1
m_nlp = 2+n
prob = ProblemMOI(n_nlp,m_nlp)

z_sol = solve(z_guess,prob)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1323
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       49
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       27
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

49-element Vector{Float64}:
 -4.970596229551274e-38
 -1.1221567232645722
 -2.087007138408125
 -2.913751812319345
 -3.618842414771345
 -4.216309786705357
 -4.718043325029628
 -5.134027140286344
 -5.472539098297339
 -5.740315398973403
 -5.942684603954684
 -6.083673747511781
 -6.166088431447457
  ⋮
 -0.0637007803012361
  0.07736787086139874
  0.21997601212027215
  0.36696155684682036
  0.5212491133580992
  0.6859095498418172
  0.8642191081524913
  1.059726066229793
  1.276321015277974
  1.5183139037807778
  1.7905205166355929
  2.0983571458148402

In [58]:
#Plot Solution
y_sol = z_sol[1:n]
plot(y_sol)